<a href="https://colab.research.google.com/github/conceptree/bdda_gira/blob/main/bdda_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Projecto de Base de Dados Distribuidas Avançadas

## Autores

*   Nuno Rodrigues
*   Tiago Alves

## Introdução

Este projecto visa aplicar as metodologias dadas em aula, referentes a base de dados distribuidas, com recurso aos dados referentes ao aluguer de bicicletas na cidade de Lisboa através do programa [GIRA](https://www.gira-bicicletasdelisboa.pt/) com uma relação às condições meteorológicas presentes durante o periodo em estudo.

Neste coLab iremos precorrer os diferentes passos para a tansformação e tratamento dos dados em enface.

## Origem dos dados

- Bicicletas (GIRA) - https://dados.gov.pt/pt/datasets/gira-bicicletas-de-lisboa-historico/
- Meteorológicos - https://www.visualcrossing.com/

# Passo 1

Este primeiro passo consiste na aplicação do script responsável pelo tratamento dos dados referentes às diferentes estações de bicicletas da cidade de Lisboa.

Iremos tratar dois datasets retirados da origem, já mencionada em cima, separados com base no intervalo temporal de **2022-01-01** a **2023-02-16**.

Para este tratamento iremos aplicar as seguintes transformações:

1.   Criar a coluna ID que será proveniente do primeiro resultado de um split realizado na coluna original "desigcomercial".
2.   Criar a coluna station que será que será proveniente do segundo resultado de um split realizado na coluna original "desigcomercial" e discartando completamente o caracter usado para a divisão (ex: "-").
3. Renomeamos as colunas originais que pretendemos manter da seguinte forma:
  - 'numbicicletas': 'bikes_available',
  - 'numdocas': 'total_docks',
  - 'estado': 'state'

4. Criamos a coluna latitude cuja coordenada é retirada da coluna original "position".
5. Criamos a coluna longitude cuja coordenada é retirada da coluna original "position".
6. Criamos a coluna date que retiramos da coluna original "entity_ts".
7. Criamos a coluna tim que retiramos da coluna original "entity_ts" e formatamos como HH:MM:SS.

Corremos estas transformações para o dataset 1 e 2 de forma a termos gira_h1_cleaned_dataset.csv e gira_h2_cleaned_dataset.csv finais para o passo seguinte.

In [ ]:
import pandas as pd

# Replace 'your_dataset.csv' with the path to your actual dataset file
file_path = 'Gira_H1.csv'

# Read the dataset
df = pd.read_csv(file_path)

# Print column names to verify
print("Columns in the dataset:", df.columns.tolist())

# Ensure that the column names used below match exactly with what's printed above
# Splitting 'designcomercial' into 'ID' and 'station'
df[['ID', 'station']] = df['desigcomercial'].str.split('-', n=1, expand=True)

# Renaming columns
df.rename(columns={'numbicicletas': 'bikes_available',
                   'numdocas': 'total_docks',
                   'estado': 'state'}, inplace=True)

# Extracting latitude and longitude from 'position'
df['latitude'] = df['position'].apply(lambda x: eval(x)['coordinates'][1])
df['longitude'] = df['position'].apply(lambda x: eval(x)['coordinates'][0])

# Extracting date and time from 'entity_ts'
df['date'] = pd.to_datetime(df['entity_ts']).dt.date
df['time'] = pd.to_datetime(df['entity_ts']).dt.strftime('%H:%M:%S')

# Selecting the required columns in the specified order
df = df[['ID', 'station', 'bikes_available', 'total_docks', 'state', 'latitude', 'longitude', 'date', 'time']]

# Write the cleaned data to a new CSV file
output_file = 'gira_h1_cleaned_dataset.csv'
df.to_csv(output_file, index=False)

print(f"Cleaned dataset saved as '{output_file}'")


# Passo 2

Neste passo corremos aplicamos a concatenação dos datasets resultantes do passo 1 de forma a ficarmos com apenas um dataset referente às bicicletas chamado gira_final.csv.

In [ ]:
import pandas as pd

# Load the datasets
df_01 = pd.read_csv('gira_h1_cleaned_dataset.csv')
df_02 = pd.read_csv('gira_h2_cleaned_dataset.csv')

# Concatenate the datasets
concatenated_df = pd.concat([df_01, df_02])

# Optionally, sort by date if the datasets are not already sorted
# Replace 'date_column_name' with the name of your date column
concatenated_df.sort_values(by='date', inplace=True)

# Reset the index of the concatenated dataset
concatenated_df.reset_index(drop=True, inplace=True)

# Save the concatenated dataset to a new CSV file
concatenated_df.to_csv('gira_final.csv', index=True, index_label="index")

print("Concatenated dataset saved.")


# Passo 3

Agora só nos falta criar as colunas referentes aos indicadores meteorológicos que pretendemos para este estudo, sendo os mesmos:

*   Temperatura
*   Precipitação
*   Temperatura que se faz sentir

Este indicadores serão retirados do dataset Lx_Meteo.csv que por sua vez inclui todos os dados meteorológicos da cidade de Lisboa dentro do intervalo de tempo em estudo.

Após este passo iremos terminar com o que será o nosso dataset final que vai pelo nome de final_dataset.csv.

In [ ]:
import pandas as pd

# Assuming the concatenated dataset is available at the given path
concatenated_path = 'gira_final.csv'  # Update this with the actual file path
concatenated_df = pd.read_csv(concatenated_path)

# Load the meteorological data
meteo_path = './Lx_Meteo.csv'
meteo_df = pd.read_csv(meteo_path)

# Ensure date formats are aligned (modify if necessary)
concatenated_df['date'] = pd.to_datetime(concatenated_df['date'])
meteo_df['datetime'] = pd.to_datetime(meteo_df['datetime'])

# Merge the datasets on the date column
merged_df = pd.merge(concatenated_df, meteo_df[['datetime', 'feelslike', 'temp', 'precip']],
                     left_on='date', right_on='datetime', how='left')

# Drop the extra 'datetime' column if not needed
merged_df.drop('datetime', axis=1, inplace=True)

# Save the merged dataset
merged_df.to_csv('final_dataset.csv', index=False)

print("Final merged dataset saved.")


# Conclusão

Após todas as transformações realizadas nos passos em cima mencionados, temos então, o que serão os dados que utilizaremos como base de estudo para este projecto.

Dos mesmos podemos então abordar algunas das seguintes análises e áreas negócio:


### Análise de Procura:

* Impacto do Clima no Uso de Bicicletas: Analisar como diferentes condições climáticas (como temperatura, precipitação e sensação térmica) afetam o uso de bicicletas. Isso pode ajudar a prever a procura em diferentes condições climáticas e diferentes pontos da cidade.

* Tendências Sazonais: Examinar padrões de uso em diferentes estações do ano para identificar períodos de pico e baixa procura.

### Eficiência Operacional:

* Necessidades de Reequilíbrio: Identificar estações que frequentemente ficam sem bicicleta e correlacionar isso com padrões climáticos para otimizar a redistribuição de bicicletas.

* Agendamento de Manutenção: Procurar padrões no uso de bicicletas para programar manutenções em momentos de menor demanda, possivelmente influenciados pelas condições climáticas. Isto pode ser feito com base no estado actual de cada estação.

### Análise do Comportamento do Cliente:

* Duração e Distância: Entender como as condições climáticas afetam a duração e a distância das viagens de bicicleta.

* Horários de Pico de Uso: Determinar os horários do dia em que as bicicletas são mais requisitadas em cada uma das estações e como isso pode variar com o clima.

### Estratégias de Marketing e Promoção:

* Promoções Direcionadas: Desenvolver promoções baseadas no clima, como descontos em dias ensolarados ou durante condições climáticas menos populares para aumentar o uso.

### Análise de Segurança:

* Incidentes Relacionados ao Clima: Analisar se certas condições climáticas levam a mais acidentes ou problemas de segurança.

* Visibilidade e Condições: Avaliar o impacto da visibilidade (chuva forte) na segurança do uso de bicicletas. (Aqui apesar de interessante, talvez seja necessário cruzar dados de incidentes com bicicletas)

### Impacto Ambiental:

* Reduções de Emissões: Estimar a redução nas emissões de carbono devido ao uso de bicicletas em vez de dirigir, possivelmente influenciado pelas condições climáticas. (Algo também interessante mas, novo cruzamento seria necessário com dados referentes às emissões em determinados periodos)

* Promoção do Transporte Verde: Usar os dados para promover o ciclismo como um modo de transporte sustentável, especialmente em dias claros e com baixa poluição. (Argumento provindo do ponto anterior)

### Análise Geográfica:

* Rotas e Áreas Populares: Identificar as rotas e áreas mais populares para andar de bicicleta e como as condições climáticas afetam essas escolhas.

* Oportunidades de Expansão: Usar os dados para identificar locais potenciais
para novas estações de bicicletas.

### Análise Financeira:

* Flutuações de Receita: Investigar como as condições climáticas impactam a receita do uso de bicicletas.

### Previsão:

* Modelação Preventiva: Desenvolver modelos preditivos para prever a procura de compartilhamento de bicicletas com base nas previsões meteorológicas.

* Modelos de Preços Dinâmicos: Explorar estratégias de preços dinâmicos baseadas na demanda antecipada influenciada pelas condições climáticas.

# HDFS e Distribuição

Parte importante deste projecto é agora aproveitar os dados que temos e colocar em prática a metodologias de base de dados distribuidas dadas em aula com recurso ao ecosistema Hadoop.

Para os passos seguintes recorremos ao stack Hadoop presente em https://github.com/fabiogjardim/bigdata_docker

Neste projecto vamos abordar este requisto da seguinte maneira:

1. Vamos partir os dados em três csv´s diferentes
2. Vamos colocar os dados no HDFS através dos comandos:


Aceder ao ambiente de bash do contentor do HDFS

```
 docker exec -it datanode bash
```
E executar os comandos:
```
hdfs dfs -mkdir /gira/
hdfs dfs -put /caminho/local/dos/dados /caminho/no/hdfs
```

> **_NOTA:_** O segundo comando será apenas corrido nos passos mais a frente assim que se souber o caminho da pasta local partilhada com o contentor em questão sendo que o /caminho/no/hdfs será então o já criado /gira/

Vamos primeiro dividir o nosso dataset em três partes iguais recorrrendo ao seguinte script:


In [ ]:
import pandas as pd

def split_csv(input_file, output_files):
    # Read the input CSV file
    data = pd.read_csv(input_file)

    # Calculate the size of each split
    split_size = len(data) // len(output_files)

    # Split the data and save to new files
    for i, output_file in enumerate(output_files):
        if i < len(output_files) - 1:
            data_split = data.iloc[i*split_size:(i+1)*split_size]
        else:
            # Ensure the last file includes any remaining rows
            data_split = data.iloc[i*split_size:]
        data_split.to_csv(output_file, index=False)

# Usage
input_csv = 'final_dataset.csv'
output_csvs = ['data_1.csv', 'data_2.csv', 'data_3.csv']
split_csv(input_csv, output_csvs)

Uma vez que tenhamos os nossos dados separados temos então de partilhar a sua localização na máquina local com um volume partilhado no contentor docker que temos a correr.

Para saber qual o nosso directório local onde teremos de colocar os dados de forma a serem acessíveis no contentor, temos de verificar qual o volume partilhado do mesmo, que neste caso será onde se encontra a correr o HDFS que segundo a documentação da docker que estamos a utilizar corre sobe o nome de "datanode".

Para obter essa informação precisamos apenas de correr:


```
docker inspect datanode
```

O comando em cima deverá imprimir a informação de "Mounts" onde concretamente só necessitamos das seguintes informações:


```
"Mounts": [
    {
        "Type": "bind",
        "Source": "/caminho/na/sua/maquina",
        "Destination": "/caminho/no/container",
        ...
    }
]
```

Agora temos tudo o que necessitamos para partilhar os dados transformados com o contentor do HDFS basta então copiar os mesmos para a pasta referida na "Source" e proceder aos seguintes comandos:


```
docker exec -it datanode /bin/bash
```

Para aceder ao ambiente bash do contentor em questão:



```
hdfs dfs -put /hadoop/dfs/data/* /gira/
```

Relembrando que nos primeiros passos em cima "gira" foi o nome que estabelecemos enquanto directório no HDFS.

Uma vez todos os passos em cima estejam realizados com sucesso podemos então comprovar a distribuição através do seguinte comando:

```
hdfs dfs -ls /gira/
```

Que deverá imprimir algo como:

```
-rw-r--r--   3 root supergroup  146754628 2023-12-29 16:33 /gira/data_1.csv
-rw-r--r--   3 root supergroup  148435940 2023-12-29 16:34 /gira/data_2.csv
-rw-r--r--   3 root supergroup  142260097 2023-12-29 16:34 /gira/data_3.csv
```

E para verificar a localização dos blocos de dados nos diferentes nós do cluster:

```
hdfs fsck /gira/ -files -blocks
```

```
/// OUTROS BLOCOS LISTADOS

/gira/data_1.csv 146754628 bytes, 2 block(s):  Under replicated BP-810256244-172.18.0.7-1702412100106:blk_1073742083_1262. Target Replicas is 3 but found 1 replica(s).
 Under replicated BP-810256244-172.18.0.7-1702412100106:blk_1073742084_1263. Target Replicas is 3 but found 1 replica(s).
0. BP-810256244-172.18.0.7-1702412100106:blk_1073742083_1262 len=134217728 repl=1
1. BP-810256244-172.18.0.7-1702412100106:blk_1073742084_1263 len=12536900 repl=1

/gira/data_2.csv 148435940 bytes, 2 block(s):  Under replicated BP-810256244-172.18.0.7-1702412100106:blk_1073742085_1264. Target Replicas is 3 but found 1 replica(s).
 Under replicated BP-810256244-172.18.0.7-1702412100106:blk_1073742086_1265. Target Replicas is 3 but found 1 replica(s).
0. BP-810256244-172.18.0.7-1702412100106:blk_1073742085_1264 len=134217728 repl=1
1. BP-810256244-172.18.0.7-1702412100106:blk_1073742086_1265 len=14218212 repl=1

/gira/data_3.csv 142260097 bytes, 2 block(s):  Under replicated BP-810256244-172.18.0.7-1702412100106:blk_1073742087_1266. Target Replicas is 3 but found 1 replica(s).
 Under replicated BP-810256244-172.18.0.7-1702412100106:blk_1073742088_1267. Target Replicas is 3 but found 1 replica(s).
0. BP-810256244-172.18.0.7-1702412100106:blk_1073742087_1266 len=134217728 repl=1
1. BP-810256244-172.18.0.7-1702412100106:blk_1073742088_1267 len=8042369 repl=1

/gira/in_use.lock 12 bytes, 1 block(s):  Under replicated BP-810256244-172.18.0.7-1702412100106:blk_1073742089_1268. Target Replicas is 3 but found 1 replica(s).
0. BP-810256244-172.18.0.7-1702412100106:blk_1073742089_1268 len=12 repl=1

Status: HEALTHY
 Total size:	851835484 B
 Total dirs:	9
 Total files:	181
 Total symlinks:		0
 Total blocks (validated):	184 (avg. block size 4629540 B)
 Minimally replicated blocks:	184 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	184 (100.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	3
 Average block replication:	1.0
 Corrupt blocks:		0
 Missing replicas:		368 (66.666664 %)
 Number of data-nodes:		1
 Number of racks:		1
FSCK ended at Fri Dec 29 16:38:42 UTC 2023 in 13 milliseconds


The filesystem under path '/gira' is HEALTHY
```

Neste print conseguimos comprovar a replicação dos nossos dados ainda que a com uma réplica apenas. Isto deve-se ao facto de estarmos apenas a correr esta mesma docker em apenas um Node para efeitos de demonstração.

Uma vez em ambiente de produção teriamos vários Data Nodes distribuídos por diferentes máquinas físicas ou virtuais, permitindo que o sistema mantenha várias cópias dos dados em diferentes nós para tolerância a falhas. No entanto, em um único nó, a alta disponibilidade e a tolerância a falhas não são possíveis, pois se o único nó falhar, todos os dados ficaram inacessíveis.




# Consultas via Hive

Agora falta apenas utilizarmos uma das ferramentas do stack para a criação de tabelas através dos dados inseridos no HDFS e efectuar consultas aos mesmos.

Para isto vamos utilizar o Hive através do interface web Hue.

Primeiro temos de criar a nossa tabela à qual vamos efectuar as consultas necessárioas. Para isso vamos então proceder aos seguintes passos:

1. Aceder ao Hue que corre em http://localhost:8888/hue/home/
2. Escolher o Hive através do menu de "Query" e em seguida "Editor" e depois Hive.
3. Tendo em conta que o Hive conta só encontrar ficheiros de texto nos diretórios onde devem estar os dados presentes, teremos de mover os nossos datasets para uma pasta onde isso seja garantido. Para isso só precisamos correr:

```
hdfs dfs -mkdir /gira/hive_query/
hdfs dfs -cp /gira/data_*.csv /gira/hive_query/
```

4. Criar a tabela através da seguinte query:
```
CREATE EXTERNAL TABLE bikes_data (
    `index` INT,
    ID INT,
    station STRING,
    bikes_available INT,
    total_docks INT,
    state STRING,
    latitude DOUBLE,
    longitude DOUBLE,
    `date` STRING,
    time STRING,
    feelslike DOUBLE,
    temp DOUBLE,
    precip DOUBLE
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs://namenode:8020/gira/hive_query/'
TBLPROPERTIES ("skip.header.line.count"="1");
```

5. Proceder a uma consulta simples como:

```
SELECT * FROM bikes_data
```

E devemos conseguir então visualizar todas as entradas dos nossos dados.

# Conclusão

Com todos os passos em cima podemos então validar os seguintes pontos:

## Configuração do Ambiente Hadoop

Foi estabelecido um ambiente Hadoop com um único nó, utilizando Docker. Este ambiente é adequado para desenvolvimento e testes, mas não recomendado para produção devido à incapacidade de garantia da integridade dos dados com apenas um nó.

## Verificação de Dados no HDFS

Usando o comando hdfs fsck, foi possível verificar a integridade dos arquivos no Hadoop Distributed File System (HDFS), garantindo que os dados estavam acessíveis e íntegros.

## Integração com o Hue

O Hue foi utilizado para facilitar a interação com o HDFS e o Hive. O Hue oferece uma interface gráfica que simplifica a execução de queries e a gestão de dados.

## Criação de Tabelas no Hive

Foram criadas tabelas no Hive para representar os conjuntos de dados presentes no HDFS. Isso permitiu a execução de queries SQL sobre os dados, facilitando análises e consultas.

## Resolução de Problemas com Localização de Arquivos

Inicialmente, existiram dificuldades na criação de tabelas devido à especificação incorreta da localização dos arquivos CSV no HDFS devido ao facto do Hive esperar apenas que existam ficheiros de texto nos diretórios especificados algo que não é garantido devido à criação de pastas e ficheiros pelo próprio HDFS. Após ajustes, as tabelas foram criadas com sucesso.

A estrutura de diretórios do HDFS foi modificada para organizar os arquivos CSV de forma mais eficiente, facilitando a criação de tabelas no Hive sem colocar em causa a lógica de replicação do HDFS.

## Consulta de Dados via Hive

Com as tabelas criadas, foi possível realizar consultas SQL sobre os dados dos CSVs. Isso demonstrou a integração eficaz entre o Hive e o HDFS, permitindo análises complexas de dados.

## Replicação e Gerenciamento de Dados no HDFS

Foi discutida a lógica de replicação do HDFS e como ela se aplica à cópia de arquivos dentro do sistema. Isso é crucial para entender o gerenciamento de espaço e a integridade dos dados no HDFS.

## Resultados Alcançados

É agora possível realizar análises complexas e consultas em larga escala nos dados armazenados no HDFS usando o Hive.

## Flexibilidade e Facilidade de Uso

A utilização do Hue para interagir com o Hadoop e o Hive trouxe uma camada adicional de facilidade e flexibilidade para o gerenciamento de dados.

## Compreensão da Gestão de Dados no HDFS

Os passos realizados proporcionaram uma compreensão prática de como gerenciar e organizar dados dentro de um ambiente Hadoop.

### Resumo:

Este processo demonstrou como configurar e utilizar um ambiente Hadoop com Docker, realizar a integração com ferramentas como o Hue e o Hive, e como gerenciar dados dentro do HDFS. Além disso, destacou a importância de entender a estrutura de diretórios e a replicação no HDFS para a gestão eficaz de grandes volumes de dados.